In [2]:
import math
from pprint import pprint
from math import log
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import matrix
import mathematics
from importlib import reload

for m in (arrays, trees, lists, graphs, search, strings, matrix, mathematics):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import matrix as mt
import mathematics as m

print()

In [8]:
class BitTrie:
    """A trie sorting integers with nodes splitting for each bit.

    Each node in the trie is just an array of two, with one cell for 0 and the other for 1.
    The leaf nodes are at the same distance of max `bits`.
    The leaf nodes also store the actual number as data.
    """

    def __init__(self, bits: int = 32):
        """Initialize the `BitTrie`.

        Args:
            bits (int, optional): Maximum number of bits for integers stored. Defaults to 32.
        """
        self.root = [0, 0]
        self.bits = bits

    def insert(self, num: int, data=0):
        "Insert `num` into the trie. Extra `data` can be stored with it."
        n: list = self.root
        for i in reversed(range(self.bits)):
            r = (num >> i) & 1
            n[r] = n[r] or [0, 0]
            n = n[r]
        # Store the `num` in the last node.
        n[:] = [num, data]

    def nearest(self, num: int):
        "Find the nearest number to `num`. Returns the number plus additional data."
        n: list = self.root
        for i in reversed(range(self.bits)):
            r = (num >> i) & 1
            # First lookup a similar bit,
            # otherwise lookup the opposite bit
            n = n[r] or n[1 - r]
        # The numeric value and data are stored in the last node.
        return n


def max_xor_sub_array(a: list) -> Tuple[int, list]:
    "For an array `a` of integers, find the largest XOR value of a sub-array."
    # The idea is to generate a cumulative XOR value,
    # and find the previous cumulative value which would result in the highest
    # XOR if applied to the current XOR value.
    #
    # Finding the previous XOR value can be done by maintaining a trie
    # with values stored so far. The lookup is done using the negative
    # current running xor value as this would give us the largest
    # result (i.e. ~v ^ v is the largest possible result.)
    # In the trie we look for the nearest value stored so far, descending
    # the tree by taking path matching the bits in the `~v` if possible.
    #
    # Build a bit trie using the longest bit_length.
    trie = BitTrie(bits=max(map(int.bit_length, a)))
    # Insert 0 as the starter for the cumulative xor value.
    trie.insert(0, -1)
    mx = (0, 0, 0)  # maximum result so far
    cx = 0  # cummulative xor value for the array
    for i, e in enumerate(a):
        cx ^= e
        # First insert the new value.
        trie.insert(cx, i)
        # Then lookup the nearest `~cx` value in the trie.
        # By xoring cx and the the nearest negative,
        # we get the maximum result for this array prefix.
        # We can update the maximum result `mx` for the overall array.
        nearest = trie.nearest(~cx)
        mx = max(mx, (cx ^ nearest[0], nearest[1]+1, i+1))
    return mx[0], a[mx[1] : mx[2]]

In [10]:
max_xor_sub_array([1, 2, 3, 4, 5])

(7, [3, 4])

In [204]:
n = 10
n.bit_length()

4